# Fruits Detection with YOLO11 and OAK-1 Lite

This notebook contains:
- Custom scripts developed by Johnatan Valencia.
- Scripts adapted from the tutorial by Evan Juras.

References are provided below.

In [ ]:
# ------------------------------
# ENVIRONMENT SETUP (common)
# ------------------------------

!pip install ultralytics depthai blobconverter onnx

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

!nvidia-smi

### Dataset Preparation

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/MyDrive/PDI/testing/fruits_detectionYOLO/"

!cp "/content/gdrive/MyDrive/PDI/testing/fruits_detectionYOLO/data.zip" /content/

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
# ------------------------------
# SCRIPT BASED ON EVAN JURAS
# Source: https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb#scrollTo=PooP5Vjsg2Jn
# Adapted for YOLO11
# ------------------------------

import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

✍️ ### Custom Export Functions
The following cells were developed specifically for this project.


In [ ]:
# ------------------------------
# CUSTOM SCRIPTS - by Johnatan Valencia
# ------------------------------

# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# ------------------------------
# CUSTOM SCRIPTS - by Johnatan Valencia
# ------------------------------


# This takes forever for some reason, you can also just download the model from the sidebar
from google.colab import files

files.download('/content/my_model.zip')

In [ ]:
# ------------------------------
# CUSTOM SCRIPTS - by Johnatan Valencia
# ------------------------------


!yolo export model=runs/detect/train/weights/best.pt format=onnx opset=12 imgsz=640 simplify=True dynamic=False

!python -m blobconverter \
--onnx-model runs/train/exp/weights/best.onnx \
--shaves 4 \
--output-dir /content \
--data-type FP16 \
--version 2022.1 \
--optimizer-params="--reverse_input_channels --input_shape=[1,3,640,640]"

!mkdir -p /content/my_model

# copy onnx file
!cp /content/runs/detect/train/weights/best.onnx /content/my_model/best.onnx

# compress
!zip /content/my_model.zip /content/my_model/best.onnx

# download file
from google.colab import files
files.download('/content/my_model.zip')

!mkdir -p /content/my_model

# better location onnx file
!cp /content/runs/detect/train/weights/best.onnx /content/my_model/best.onnx

# compress for downloading
!zip /content/my_model.zip /content/my_model/best.onnx

# download file
from google.colab import files
files.download('/content/my_model.zip')